# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [86]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col
from pyspark.sql.window import Window

#### Function definitions

In [2]:
def missing_values_count(df):
    return df.select([F.count(F.when(F.isnan(c) | F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

In [3]:
# Start a Spark Session
spark = SparkSession.builder \
    .appName("InterviewChallenge") \
    .master("local[2]") \
    .getOrCreate()

#     .config("spark.executor.memory", "2g") \

In [48]:
# Read input data to spark dataframe
flights_path = 'Data/flights.csv' 
airlines_path = 'Data/airlines.csv'
airports_path = 'Data/airports.csv'

df = spark.read.options(header=True).csv(flights_path)  # main dataset with flights data
# df = spark.read.options(header=True, inferSchema=True).csv(flights_path)  # main dataset with flights data
airlines_df = spark.read.options(header=True, inferSchema=True).csv(airlines_path)  # data dict with airlines
airports_df = spark.read.options(header=True, inferSchema=True).csv(airports_path)  # data dict with airports

In [5]:
df.printSchema()

root
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)
 |-- DAY_OF_WEEK: string (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: string (nullable = true)
 |-- TAXI_OUT: string (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: string (nullable = true)
 |-- ELAPSED_TIME: string (nullable = true)
 |-- AIR_TIME: string (nullable = true)
 |-- DISTANCE: string (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: string (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: string (nullable = true)
 |-- D

In [6]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,YEAR,string
1,MONTH,string
2,DAY,string
3,DAY_OF_WEEK,string
4,AIRLINE,string
5,FLIGHT_NUMBER,string
6,TAIL_NUMBER,string
7,ORIGIN_AIRPORT,string
8,DESTINATION_AIRPORT,string
9,SCHEDULED_DEPARTURE,string


In [7]:
df.show(20)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

In [8]:
# Dataframe shape
print("Dataframe row count: ", df.count())
print("Dataframe columns count: ", len(df.columns))

Dataframe row count:  5819079
Dataframe columns count:  31


In [9]:
# Column statistics
# df.describe().toPandas()

In [10]:
missing_values_count(df)

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+-

##### Airports data dictionary

In [11]:
airports_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



In [12]:
# DTypes in pd formatting
pd.DataFrame(airports_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRPORT,string
2,CITY,string
3,STATE,string
4,COUNTRY,string
5,LATITUDE,double
6,LONGITUDE,double


In [13]:
airports_df.show(20)

+---------+--------------------+-------------+-----+-------+--------+----------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|
+---------+--------------------+-------------+-----+-------+--------+----------+
|      ABE|Lehigh Valley Int...|    Allentown|   PA|    USA|40.65236|  -75.4404|
|      ABI|Abilene Regional ...|      Abilene|   TX|    USA|32.41132|  -99.6819|
|      ABQ|Albuquerque Inter...|  Albuquerque|   NM|    USA|35.04022|-106.60919|
|      ABR|Aberdeen Regional...|     Aberdeen|   SD|    USA|45.44906| -98.42183|
|      ABY|Southwest Georgia...|       Albany|   GA|    USA|31.53552| -84.19447|
|      ACK|Nantucket Memoria...|    Nantucket|   MA|    USA|41.25305| -70.06018|
|      ACT|Waco Regional Air...|         Waco|   TX|    USA|31.61129| -97.23052|
|      ACV|      Arcata Airport|Arcata/Eureka|   CA|    USA|40.97812|-124.10862|
|      ACY|Atlantic City Int...|Atlantic City|   NJ|    USA|39.45758| -74.57717|
|      ADK|        Adak Airp

In [14]:
# Dataframe shape
print("Dataframe row count: ", airports_df.count())
print("Dataframe columns count: ", len(airports_df.columns))

Dataframe row count:  322
Dataframe columns count:  7


In [15]:
# Column statistics
airports_df.describe().toPandas()

,summary,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,count,322,322,322,322,322,319,319
1,mean,None,None,None,None,None,38.981243918495295,-98.37896445141064
2,stddev,None,None,None,None,None,8.616735581018041,21.523492046498102
3,min,ABE,Aberdeen Regional Airport,Aberdeen,AK,USA,13.48345,-176.64603
4,max,YUM,Yuma International Airport,Yuma,WY,USA,71.28545,-64.79856


In [16]:
missing_values_count(airports_df)

+---------+-------+----+-----+-------+--------+---------+
|IATA_CODE|AIRPORT|CITY|STATE|COUNTRY|LATITUDE|LONGITUDE|
+---------+-------+----+-----+-------+--------+---------+
|        0|      0|   0|    0|      0|       3|        3|
+---------+-------+----+-----+-------+--------+---------+



#### Airlines data dictionary

In [17]:
airlines_df.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRLINE: string (nullable = true)



In [18]:
# DTypes in pd formatting
pd.DataFrame(airlines_df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,IATA_CODE,string
1,AIRLINE,string


In [19]:
airlines_df.show(20)

+---------+--------------------+
|IATA_CODE|             AIRLINE|
+---------+--------------------+
|       UA|United Air Lines ...|
|       AA|American Airlines...|
|       US|     US Airways Inc.|
|       F9|Frontier Airlines...|
|       B6|     JetBlue Airways|
|       OO|Skywest Airlines ...|
|       AS|Alaska Airlines Inc.|
|       NK|    Spirit Air Lines|
|       WN|Southwest Airline...|
|       DL|Delta Air Lines Inc.|
|       EV|Atlantic Southeas...|
|       HA|Hawaiian Airlines...|
|       MQ|American Eagle Ai...|
|       VX|      Virgin America|
+---------+--------------------+



In [20]:
# Dataframe shape
print("Dataframe row count: ", airlines_df.count())
print("Dataframe columns count: ", len(airlines_df.columns))

Dataframe row count:  14
Dataframe columns count:  2


In [21]:
# Column statistics
airlines_df.describe().toPandas()

,summary,IATA_CODE,AIRLINE
0,count,14,14
1,mean,None,None
2,stddev,None,None
3,min,AA,Alaska Airlines Inc.
4,max,WN,Virgin America


In [22]:
missing_values_count(airlines_df)

+---------+-------+
|IATA_CODE|AIRLINE|
+---------+-------+
|        0|      0|
+---------+-------+



### Step 2: Explore and Assess the Data
#### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.



In [23]:
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: DLL load failed while importing _imaging: The specified module could not be found.

In [ ]:
fig = plt.figure(figsize=(25, 15))
st = fig.suptitle("Distribution of Features", fontsize=50, verticalalignment="center")
for col, num in zip(df.toPandas().describe().columns, range(1,11)):
    ax = fig.add_subplot(3,4, num)
    ax.hist(df.toPandas[col])
    plt.grid(False)
    plt.xticks(ratation=45, fontsize=20)
    plt.yticks(fontsize=15)
    plt.title(col.upper(), fontsize=20)
    
plt.tight_layout()
st.set_y(0.95)
fig.subplots_adjust(top=0.85, hspace=0.4)
plt.show()


#### Cleaning Steps
Document steps necessary to clean the data

- converting to integers
- converting dates to datetime
- ~~converting time to time~~
- convert DIVERTED CANCELLED to Boolean

In [49]:
# Converting to integers 

int_cols = ['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'DEPARTURE_DELAY', 'TAXI_OUT', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'TAXI_IN', 
            'ARRIVAL_DELAY', 'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
for c in int_cols :
    df = df.withColumn(c, F.col(c).cast("int"))

In [50]:
# Converting date

df = df.withColumn('DATE', F.make_date('YEAR', 'MONTH', 'DAY'))

In [26]:
# Converting time
# df1 = df.withColumn('SCHEDULED_DEPARTURE', F.make_timestamp(hour=F.substring('SCHEDULED_DEPARTURE', 1, 2),min=F.substring('SCHEDULED_DEPARTURE', 3, 2)))
# Converting time
# df1 = df.withColumn('SCHEDULED_DEPARTURE', F.to_timestamp('SCHEDULED_DEPARTURE', format='HHmm'))

In [51]:
# Converting booleans

df = df.withColumn('DIVERTED', F.col('DIVERTED').cast('boolean'))
df = df.withColumn('CANCELLED', F.col('CANCELLED').cast('boolean'))

In [28]:
df.show()

+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+----------+
|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|      DATE|
+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-

In [77]:
df.filter((df['AIRLINE'] == 'AS') & (df['FLIGHT_NUMBER'] == 98)).tail(50)

[Row(YEAR=2015, MONTH=10, DAY=12, DAY_OF_WEEK=1, AIRLINE='AS', FLIGHT_NUMBER='98', TAIL_NUMBER='N760AS', ORIGIN_AIRPORT='10299', DESTINATION_AIRPORT='14747', SCHEDULED_DEPARTURE='1630', DEPARTURE_TIME='1626', DEPARTURE_DELAY=-4, TAXI_OUT=8, WHEELS_OFF='1634', SCHEDULED_TIME=199, ELAPSED_TIME=223, AIR_TIME=208, DISTANCE=1448, WHEELS_ON='2102', TAXI_IN=7, SCHEDULED_ARRIVAL='2049', ARRIVAL_TIME='2109', ARRIVAL_DELAY=20, DIVERTED=False, CANCELLED=False, CANCELLATION_REASON=None, AIR_SYSTEM_DELAY=20, SECURITY_DELAY=0, AIRLINE_DELAY=0, LATE_AIRCRAFT_DELAY=0, WEATHER_DELAY=0, DATE=datetime.date(2015, 10, 12)),
 Row(YEAR=2015, MONTH=10, DAY=13, DAY_OF_WEEK=2, AIRLINE='AS', FLIGHT_NUMBER='98', TAIL_NUMBER='N317AS', ORIGIN_AIRPORT='10299', DESTINATION_AIRPORT='14747', SCHEDULED_DEPARTURE='1630', DEPARTURE_TIME='1628', DEPARTURE_DELAY=-2, TAXI_OUT=10, WHEELS_OFF='1638', SCHEDULED_TIME=199, ELAPSED_TIME=199, AIR_TIME=183, DISTANCE=1448, WHEELS_ON='2041', TAXI_IN=6, SCHEDULED_ARRIVAL='2049', ARRIVA

#### Post cleaning statistics

In [30]:
df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: string (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: string (nullable = true)
 |-- DEPARTURE_TIME: string (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: string (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: string (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: string (nullable = true)
 |-- ARRIVAL_TIME: string (nullable = true)
 |-- ARRIVAL_DELAY: integer (nullable = 

In [52]:
# DTypes in pd formatting
pd.DataFrame(df.dtypes, columns=['Column', 'Dtype'])

,Column,Dtype
0,YEAR,int
1,MONTH,int
2,DAY,int
3,DAY_OF_WEEK,int
4,AIRLINE,string
5,FLIGHT_NUMBER,string
6,TAIL_NUMBER,string
7,ORIGIN_AIRPORT,string
8,DESTINATION_AIRPORT,string
9,SCHEDULED_DEPARTURE,string


### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

##### FLIGHTS table fact
- ID PK serial PK
- FLIGHT_ID serial FK SCHEDULE
- DEPARTURE_TIME
- DEPARTURE_DELAY int
- TAXI_OUT int
- WHEELS_OFF
- ELAPSED_TIME
- AIR_TIME
- WHEELS_ON
- TAXI_IN
- ARRIVAL_TIME
- ARRIVAL_DELAY
- DIVERTED
- CANCELLED
- CANCELLATION_REASON
- AIR_SYSTEM_DELAY
- SECURITY_DELAY
- AIRLINE_DELAY
- LATE_AIRCRAFT_DELAY
- DATE


##### SCHEDULE table dim
- FLIGHT_ID serial PK composite key AIRLINE FLIGHT NUMBER SCHEDULED DEPARTURE SCHEDULED TIME
- AIRLINE str
- FLIGHT_NUMBER str
- TAIL_NUMBER str
- ORIGIN_AIRPORT
- DESTINATION_AIRPORT
- SCHEDULED_DEPARTURE
- SCHEDULED_TIME
- DISTANCE
- SCHEDULED_ARRIVAL

##### AIRPORTS table dim
- IATA_CODE serial PK
- AIRPORT str
- CITY_ID serial FK CITIES
- LATITUDE
- LONGITUDE

##### AIRLINES table dim
- IATA_CODE serial PK
- AIRLINE str

##### CITIES table dim
- ID serial PK
- CITY str UNIQUE
- STATE str
- COUNTRY str

#### TIME table dim
- DATE date PK
- YEAR
- MONTH
- DAY
- DAY_OF_WEEK

![Image](Img\data-org-chart.png)

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

1. Create a new column ID for storing primary key of FLIGHTS fact table by generating serial for each row in df
2. Create a new columns FLIGHTS_ID for storing primary key of SCHEDULE dim table by creating composite key by concating AIRLINE, FLIGHT_NUMBER, SCHEDULED_DEPARTURE, SCHEDULED_ARRIVAL
3. Create a FLIGHTS fact table by selecting columns into flights_df dataframe
4. Save FLIGHTS table in PARQUET format partitioning by YEAR > MONTH > DAY
5. Create SCHEDULE dim table by selecting appropriate columns with unique FLIGHT_IDs into schedule_df dataframe
6. Save SCHEDULE table in PARQUET format
7. Create CITY dim table by selecting appropriate columns with unique composite key CITY, STATE, COUNTRY and generate CITY_ID
8. Save CITY dim table as parquet partitioning by COUNTRY, STATE
9. Create AIRPORTS dim table by selecting appropriate columns 
10. Save AIRPORTS dim table in PARQUET format partitioning by first letter of IATA_CODE
11. Create AIRLINES dim table by slecting appropriate columns paritioning by first letter of IATA_CODE


### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [83]:
# Generate ID columns

df = df.withColumn('ID', F.monotonically_increasing_id())
df = df.withColumn('FLIGHT_ID', F.concat(col('AIRLINE'), col('FLIGHT_NUMBER'), col('SCHEDULED_DEPARTURE'), col('SCHEDULED_ARRIVAL')))

## 

# w = Window().partitionBy('CITY', 'STATE', 'COUNTRY')
airports_df = airports_df.withColumn('CITY_ID', F.monotonically_increasing_id())

# FLIGHTS fact table

# flights_df = df.select("")

Traceback (most recent call last):
  File "c:\Users\Jakub.Pitera\Anaconda3\envs\udacity\Lib\site-packages\pyspark\serializers.py", line 458, in dumps
    return cloudpickle.dumps(obj, pickle_protocol)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jakub.Pitera\Anaconda3\envs\udacity\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 73, in dumps
    cp.dump(obj)
  File "c:\Users\Jakub.Pitera\Anaconda3\envs\udacity\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 602, in dump
    return Pickler.dump(self, obj)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jakub.Pitera\Anaconda3\envs\udacity\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 692, in reducer_override
    return self._function_reduce(obj)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Jakub.Pitera\Anaconda3\envs\udacity\Lib\site-packages\pyspark\cloudpickle\cloudpickle_fast.py", line 565, in _function_reduce
    return self._dynamic_function_red

PicklingError: Could not serialize object: IndexError: tuple index out of range

+--------------+
|     FLIGHT_ID|
+--------------+
|  AS9800050430|
|AA233600100750|
| US84000200806|
| AA25800200805|
| AS13500250320|
| DL80600250602|
| NK61200250526|
|US201300300803|
|AA111200300545|
|DL117300300711|
|DL233600300523|
|AA167400350803|
|DL143400350609|
|DL232400400615|
|DL244000400549|
| AS10800450509|
|DL156000450515|
|UA119700480626|
| AS12200500525|
|DL167000500603|
+--------------+
only showing top 20 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [ ]:
# Perform quality checks here

#### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

#### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.